In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import warnings
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
import math
from konlpy.tag import Twitter
from nltk.tag import pos_tag
from nltk.tag import untag
from nltk import Text
from matplotlib import pyplot as plt
from nltk import FreqDist
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from glob import glob
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/kimEn/Desktop", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome('chromedriver.exe', options=options)

driver.get('https://paperswithcode.com/sota')

In [39]:
url_to_first=[] 
url_to=driver.find_elements(By.CLASS_NAME,'col-md-12 > h4 > a')
for url1 in url_to:
    url_to_first.append(url1.get_attribute('href'))

In [6]:
card_url=[]
see_all_url=[]

for i in range(len(url_to_first)):
    driver.get(url_to_first[i]) 
    
    card_url_elem=driver.find_elements(By.CLASS_NAME,'card > a')
    see_all_elem = driver.find_elements(By.CLASS_NAME,'sota-all-tasks > a')
    
    for c in card_url_elem:
        if c.get_attribute('href') not in card_url:
            card_url.append(c.get_attribute('href'))

    for s in see_all_elem:          
            see_all_url.append(s.get_attribute('href'))            


In [8]:
for i in range(len(see_all_url)):
    driver.get(see_all_url[i])
    card_url_elem2=driver.find_elements(By.CLASS_NAME,'card > a')

    for c2 in card_url_elem2:
        if c2.get_attribute('href') not in card_url:
            card_url.append(c2.get_attribute('href'))

save_url=card_url

KeyboardInterrupt: 

In [9]:
# 논문 제목, 상세 url 수집
# classname>css section
import collections

title = []
url = []
pdf=[]
classification1=[]
classification2=[]

cla_dict=collections.defaultdict(list)
pdf_cla_dict={}

for i in range(len(card_url)): 
    driver.get(card_url[i])
    
    Tlist=driver.find_elements(By.CLASS_NAME,'col-lg-9.item-content > h1 > a')
    classification1_elem=driver.find_elements(By.CLASS_NAME,'dataset-header > a')
    classification2_elem=driver.find_elements(By.CLASS_NAME, 'artefact-header > h1')
        
        
    for cla1 in classification1_elem:
        cla_item=cla1.get_attribute('href').split('/')[-1].replace('-',' ')
        classification1.append(cla_item)

        for cla2 in classification2_elem:
            classification2.append(cla2.text)
            cla_dict[cla_item].append(cla2.text)
            
            
    for i in Tlist:
        if i.get_attribute not in url and i.text not in title: 
            title.append(i.text)
            url.append(i.get_attribute('href'))

            

KeyboardInterrupt: 

In [10]:
url_cla_dict={}

for i in range(len(card_url)): 
    driver.get(card_url[i])
    classification2_elem=driver.find_elements(By.CLASS_NAME, 'artefact-header > h1')
    Tlist=driver.find_elements(By.CLASS_NAME,'col-lg-9.item-content > h1 > a')
        
    for cla2 in classification2_elem:
        for i in Tlist:
            url_cla_dict[i.text]=cla2.text

KeyboardInterrupt: 

In [27]:
#pdf_url 수집
for i in range(len(url)):    
        
    driver.get(url[i])
    pdf_url=driver.find_elements(By.CLASS_NAME,'badge.badge-light')[0].get_attribute('href')
    
    if pdf_url not in pdf:
        pdf.append(pdf_url)

In [28]:
df=pd.DataFrame(columns=['title','url','pdf'])

df['title']=title
df['url']=url
df['pdf']=pdf

In [29]:
#내용저장 csv
df.to_csv("theory.csv",header=True, index=False)
df_theory=pd.read_csv('theory.csv')
df_theory

,title,url,pdf
0,Deep Residual Learning for Image Recognition,https://paperswithcode.com/paper/deep-residual...,https://arxiv.org/pdf/1512.03385v1.pdf
1,Very Deep Convolutional Networks for Large-Sca...,https://paperswithcode.com/paper/very-deep-con...,https://arxiv.org/pdf/1409.1556v6.pdf
2,MobileNets: Efficient Convolutional Neural Net...,https://paperswithcode.com/paper/mobilenets-ef...,https://arxiv.org/pdf/1704.04861v1.pdf
3,Densely Connected Convolutional Networks,https://paperswithcode.com/paper/densely-conne...,https://arxiv.org/pdf/1608.06993v5.pdf
4,CSPNet: A New Backbone that can Enhance Learni...,https://paperswithcode.com/paper/cspnet-a-new-...,https://arxiv.org/pdf/1911.11929v1.pdf
...,...,...,...
355,A 64mW DNN-based Visual Navigation Engine for ...,https://paperswithcode.com/paper/a-64mw-dnn-ba...,https://arxiv.org/pdf/1805.01831v4.pdf
356,A water-obstacle separation and refinement net...,https://paperswithcode.com/paper/a-water-obsta...,https://arxiv.org/pdf/2001.01921v1.pdf
357,Socially Aware Motion Planning with Deep Reinf...,https://paperswithcode.com/paper/socially-awar...,https://arxiv.org/pdf/1703.08862v2.pdf
358,Intention-Net: Integrating Planning and Deep L...,https://paperswithcode.com/paper/intention-net...,https://arxiv.org/pdf/1710.05627v2.pdf


In [30]:
#os로 파일 다운로드
pdf_url = df['pdf'].values.tolist()
pdf_file = [i.split('/')[-1] for i in df['pdf'].values if 'arxiv' in i]

print(f'전체 파일 수: {len(pdf_url)}')

#enumerate -> index / for n, ( , ) = n=index, ( , )=tuple
#start /b =>os background실행
for n, (url, file) in enumerate(zip(pdf_url, pdf_file)):
    print(n)
    download_path = f'C:/Users/kimEn/Downloads/pdf_download'
    os.system(f'curl "{url}" --output "{download_path}/{file}"')

전체 파일 수: 360
0
1
2


KeyboardInterrupt: 

In [31]:
#오류제외 파일변환

for i in tqdm(pdf_file):    
    try:
        text = pdf_to_txt(f'{download_path}/{i}')
        txt_file_name = '.'.join(i.split('.')[:-1])
        with open(f'{download_path}/txt/{txt_file_name}.txt', 'w', encoding='utf-8') as f:
            f.write(text)
    except Exception as e:
        print(e, i)

  0%|          | 0/351 [00:00<?, ?it/s]

name 'pdf_to_txt' is not defined 1512.03385v1.pdf
name 'pdf_to_txt' is not defined 1409.1556v6.pdf
name 'pdf_to_txt' is not defined 1704.04861v1.pdf
name 'pdf_to_txt' is not defined 1608.06993v5.pdf
name 'pdf_to_txt' is not defined 1911.11929v1.pdf
name 'pdf_to_txt' is not defined 1801.04381v4.pdf
name 'pdf_to_txt' is not defined 1610.02391v4.pdf
name 'pdf_to_txt' is not defined 1905.11946v5.pdf
name 'pdf_to_txt' is not defined 1512.00567v3.pdf
name 'pdf_to_txt' is not defined 2010.11929v2.pdf
name 'pdf_to_txt' is not defined 1503.02531v1.pdf
name 'pdf_to_txt' is not defined 1908.08962v2.pdf
name 'pdf_to_txt' is not defined 1910.01108v4.pdf
name 'pdf_to_txt' is not defined 2006.04558v6.pdf
name 'pdf_to_txt' is not defined 1710.11063v3.pdf
name 'pdf_to_txt' is not defined 2004.09813v2.pdf
name 'pdf_to_txt' is not defined 1606.07947v4.pdf
name 'pdf_to_txt' is not defined 1612.03928v3.pdf
name 'pdf_to_txt' is not defined 1909.10351v5.pdf
name 'pdf_to_txt' is not defined 1905.09717v5.pdf
n

In [32]:

def get_wordnet_pos(tagged_pos):
    for pos in ['V', 'N', 'J', 'R']:
        if tagged_pos.startswith(pos):
            return pos.lower() if pos != 'J' else 'a'
    return None

lemm = WordNetLemmatizer()
word_txt={}

for txt_file in tqdm(glob('./Downloads/pdf_download/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:
        txt = f.read().lower().replace('-\n', '')
        
        
    # 토크나이저
        re_tokenizer = RegexpTokenizer('[a-zA-Z]{2,}')

        # 토큰화
        word_tokens = re_tokenizer.tokenize(txt)

        # 불용어(stopwords) 제거
        stop_words = stopwords.words('english')
        stop_words.append('cid')
        word_tokens = [w for w in word_tokens if w not in stop_words]

        #stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english')
        word_tokens = [stemmer.stem(w) for w in word_tokens]

        word_tokens = pos_tag(word_tokens)
        word_tokens = [(w, get_wordnet_pos(tag)) for w, tag in word_tokens if get_wordnet_pos(tag) != None]
        word_tokens = [lemm.lemmatize(word, pos=tag) for word, tag in word_tokens]
        
        if word_txt.values() == None:
            for f in FreqDist(word_tokens):
                word_txt[f]=txt_file



  0%|          | 0/90 [00:00<?, ?it/s]

In [33]:
save_keywords=FreqDist(word_tokens)

In [35]:
#스니펫 안나오는것 -> 관련질문

#{f}가 제일먼저 나오는 text, 관련질문 맨 위에 있는 거 가져오기 ->클릭해야 보임
#관련 내용 부분

test_word=pd.read_csv('keywords.csv')
test_words=test_word['0']
terminologies={}

explains=[]
explains_url=[]


#for feq in FreqDist(word_tokens):
for feq in test_words:

    driver.get('https://www.google.com')
    search_elem=driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi')

    # 검색
    search_elem.clear()
    search_elem.send_keys(f'what is {feq} of deep learning')
    driver.find_element(By.CLASS_NAME, 'gNO89b').submit()

    related_question_elem=driver.find_elements(By.CLASS_NAME, 'Wt5Tfe') #mlist
    each_question_elem=driver.find_elements(By.CLASS_NAME, 'wQiwMc.ygGdYd.related-question-pair') #clist
    href_elem=driver.find_elements(By.CLASS_NAME, 'yuRUbf > a')#hreflist 
    explain_question_elem=driver.find_elements(By.CLASS_NAME,'LGOjhe')
    explain_href_elem=driver.find_elements(By.CLASS_NAME, 'yuRUbf > a')
    each_question_elem2=driver.find_elements(By.CLASS_NAME, 'wQiwMc.ygGdYd.related-question-pair')
    
    explain_elem=driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div')        
    explain_href_elem=driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div > div > div > a')

    def click_questions():
        
        """ 관련 질문 클릭하는 함수        
        click_number=클릭 횟수 (많이 클릭할 필요가 없어 제한)"""
        
        for click_number,questions in enumerate(each_question_elem):
            if click_number < 1:
                questions.click()

    for rqe in related_question_elem:
        if rqe.is_displayed:
            click_questions()
    
    
        for explain in explain_question_elem:
            explains.append(explain.text)
            for exp_url in explain_href_elem:
                explains_url.append(exp_url.get_attribute('href'))
                
                if terminologies.get(feq)==None:
                    terminologies[feq]=explain.text

    for ex in explain_elem:
        if ex.is_displayed():            
            explains.append(ex.text)
            for eh in explain_href_elem:
                explains_url.append(eh.get_attribute('href'))
                if terminologies.get(feq)==None:
                    time.sleep(1)
                    terminologies[feq]=e.text
                
                

        time.sleep(2)        

KeyboardInterrupt: 

In [36]:
from googletrans import Translator
from itertools import chain
from collections import defaultdict

Ttranslated={}
translate_result=[]

explain_translation={}
keywords_translation={}

for i in terminologies.values():
    translator=Translator()
    result=translator.translate(i, src='en', dest='ko')
    translate_result.append(result.text)
    explain_translation[i]=result.text #원문-번역 dict
    
keywords_translation=dict(zip(keywords.keys(),translate_result)) #단어-해석 dict

test_keywords_translation = defaultdict(list)

for k, v in chain(keywords.items(), keywords_translation.items()): #dict에서 중복되는거 리스트로 겹치게
    test_keywords_translation[k].append(v)


df2=pd.DataFrame.from_dict(dict(test_keywords_translation), orient='index')
df2

df2=pd.DataFrame.from_dict(dict(enumerate(test_keywords_translation.items())), orient='index')
df2

df2.to_csv("keywords.csv",header=True, index=True, encoding='utf-8')



In [37]:
# keywords_all=pd.read_csv('keywords.csv', index = False)#header=None)
# keywords_all.rename(columns={'Unnamed: 0':'인덱스(빈도수)','0':'단어', '1':'의미/해석'}, inplace=True )
keywords_all=pd.read_csv('keywords.csv', index_col=0)
keywords_all.rename(columns={'0':'단어', '1':'의미/해석'}, inplace=True )
keywords_all

,단어,의미/해석
0,0,"['procedur', '기본 딥 러닝 아키텍처는 입력 크기가 고정되어 있으며 이는..."
1,1,"['architectur', 'ImageNet은 계층의 각 노드가 수백 수천 개의 ..."
2,2,"['imagenet', '데이터 증강은 기존 데이터에서 새로운 데이터 포인트를 생성..."
3,3,"['augment', '딥 러닝에서 컴퓨터 모델은 이미지, 텍스트 또는 사운드에서 ..."
4,4,['model']
5,5,['differ']
6,6,['tabl']
7,7,['timm']
8,8,['epoch']
9,9,['paramet']


In [4]:
theory = pd.read_csv('theory.csv')
find_title = []
pdf_title = theory[['pdf','title']]

In [9]:
titles = pdf_title[pdf_title['pdf'].str.contains(txt_file_name, na=True)]

words_title=multiple_values_one_key(words_title,word,[titles['title']])

NameError: name 'multiple_values_one_key' is not defined

In [59]:
pdf_test={}
pdf_title.to

for txt_file in glob('./Downloads/pdf_download/txt/*.txt'):
    txt_file_name = os.path.basename(txt_file).replace('.txt','')        
    with open(txt_file, encoding='utf-8') as file:
        #titles = pdf_title[pdf_title['pdf'].str.contains(txt_file_name, na=True)]['title']
        #pdf_test[title]=pdf_title[pdf]

KeyError: "None of [Index(['https://arxiv.org/pdf/1512.03385v1.pdf',\n       'https://arxiv.org/pdf/1409.1556v6.pdf',\n       'https://arxiv.org/pdf/1704.04861v1.pdf',\n       'https://arxiv.org/pdf/1608.06993v5.pdf',\n       'https://arxiv.org/pdf/1911.11929v1.pdf',\n       'https://arxiv.org/pdf/1801.04381v4.pdf',\n       'https://arxiv.org/pdf/1610.02391v4.pdf',\n       'https://arxiv.org/pdf/1905.11946v5.pdf',\n       'https://arxiv.org/pdf/1512.00567v3.pdf',\n       'https://arxiv.org/pdf/2010.11929v2.pdf',\n       'https://arxiv.org/pdf/1503.02531v1.pdf',\n       'https://arxiv.org/pdf/1908.08962v2.pdf',\n       'https://arxiv.org/pdf/1910.01108v4.pdf',\n       'https://arxiv.org/pdf/2006.04558v6.pdf',\n       'https://arxiv.org/pdf/1710.11063v3.pdf',\n       'https://arxiv.org/pdf/2004.09813v2.pdf',\n       'https://arxiv.org/pdf/1606.07947v4.pdf',\n       'https://arxiv.org/pdf/1612.03928v3.pdf',\n       'https://arxiv.org/pdf/1909.10351v5.pdf',\n       'https://arxiv.org/pdf/1

In [26]:
for txt_file in glob('./Downloads/pdf_download/txt/*.txt'):
    txt_file_name = os.path.basename(txt_file).replace('.txt','')        
    with open(txt_file, encoding='utf-8') as file:
        titles = pdf_title[pdf_title['pdf'].str.contains(txt_file_name, na=True)]
print(titles)

                                       pdf  \
46  https://arxiv.org/pdf/2110.00476v1.pdf   

                                                title  
46  ResNet strikes back: An improved training proc...  


In [6]:
pdf_title.to_dict('records')

second_classification=pd.read_csv('second_classification.csv')
words_titles=pd.read_csv('word-titles.csv')


#cla1_cla2_title=pd.merge(left=second_classification,right=classification_without_word, how='left',on=)


for number in range(len(first_classification.columns)-1):
    for cla2 in second_classification['0']:
        cla1=classification_without_word[f'{number}'].str.contains(f'{cla2}')
        cla_cla2_title[cla2]=cla1




NameError: name 'pdf_title' is not defined

In [7]:
first_classification_all=pd.read_csv('first_classification-Copy1.csv').T

columns = first_classification_all.iloc[0].values
first_classification_all.columns = columns

first_classification_all = first_classification_all.iloc[1:]
first_classification_all

,computer vision,methodology,music,adversarial,natural language processing,playing games,medical,graphs,robots,speech,time series,reasoning,miscellaneous,audio,computer code,knowledge base
0,Image Classification,Knowledge Distillation,Image Generation,Image-to-Image Translation,Text Augmentation,Object Tracking,Knee Cartilage Defect Assessment,Knowledge Graph Embedding,3D Face Reconstruction,Speech Emotion Recognition,Trajectory Prediction,Visual Reasoning,Remote Sensing Image Classification,Synthetic Speech Detection,motion style transfer,Breast Cancer Detection
1,OOD Detection,Few-Shot Image Classification,Music Texture Transfer,Adversarial Attack,Machine Reading Comprehension,Continuous Control,Medical Image Segmentation,3D Hand Pose Estimation,Drone Controller,Keyword Spotting,Human motion prediction,Visual Commonsense Reasoning,Change detection for remote sensing images,Audio Fingerprint,Code Search,Entity Alignment
2,Fine-Grained Image Classification,Domain Adaptation,Music Information Retrieval,Real-World Adversarial Attack,Emotion Recognition in Conversation,Car Racing,Lesion Segmentation,Graph Embedding,Visual Odometry,Small-Footprint Keyword Spotting,Imputation,Visual Entailment,Segmentation Of Remote Sensing Imagery,Audio-Visual Synchronization,Code Generation,research knowledge graph population
3,Semantic Segmentation,Unsupervised Domain Adaptation,Music Source Separation,Model Posioning,Emotion-Cause Pair Extraction,SMAC,Brain Tumor Segmentation,Learning-To-Rank,Monocular Visual Odometry,Talking Face Generation,Multivariate Time Series Imputation,Abstract Argumentation,Lake Ice Monitoring,Language Identification,Source Code Summarization,Non-Intrusive Load Monitoring
4,Tumor Segmentation,Domain Generalization,Music Transcription,Website Fingerprinting Defense,Image Captioning,DQN Replay Dataset,Brain Segmentation,NMR J-coupling,3D Character Animation From A Single Photo,Speech Recognition,Sequential skip prediction,Commonsense Reasoning for RL,The Semantic Segmentation Of Remote Sensing Im...,Zero-Shot Multi-Speaker TTS,Neural Network simulation,Link Prediction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,6D Pose Estimation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,6D Pose Estimation using RGB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779,Head Pose Estimation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,Vehicle Pose Estimation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
second_classification = pd.read_csv('second_classification-Copy1.csv', index_col=0)
second_classification.drop(columns=['Unnamed: 0.1'], inplace=True)

In [9]:
cla1 = []
for second in second_classification.cla2.values:
    for first in first_classification_all.columns:
        status = 0
        if second in first_classification_all[first].values:
            cla1.append(first)
            status = 1
            break
    if not status:
        cla1.append(np.nan)

second_classification['cla1'] = cla1
second_classification

,title,cla2,cla1
0,Deep Residual Learning for Image Recognition,Retinal OCT Disease Classification,computer vision
1,Very Deep Convolutional Networks for Large-Sca...,Image Classification,computer vision
2,MobileNets: Efficient Convolutional Neural Net...,Image Classification,computer vision
3,Densely Connected Convolutional Networks,Crowd Counting,computer vision
4,CSPNet: A New Backbone that can Enhance Learni...,Real-Time Object Detection,computer vision
...,...,...,...
3250,Dynamic Distillation Network for Cross-Domain ...,cross-domain few-shot learning,computer vision
3251,DeepMatching: Hierarchical Deformable Dense Ma...,Dense Pixel Correspondence Estimation,computer vision
3252,DGC-Net: Dense Geometric Correspondence Network,Dense Pixel Correspondence Estimation,computer vision
3253,Space-Time Correspondence as a Contrastive Ran...,Dense Pixel Correspondence Estimation,computer vision


In [154]:
cla1_list=[]
cla2_list=[]

for title in pdf_title['title'].values:
    #for class2 
    if title in second_classification.title.values:
        find_cla2 = second_classification[second_classification['title'] == title]
        for cla2s in find_cla2.values:
            for cla2 in cla2s[1::3]:
                cla2_list.append(cla2)
            for cla1 in cla2s[2::3]:
                cla1_list.append(cla1)
                
pdf_title['cla1'] = cla1_list
pdf_title['cla2'] = cla2_list

In [156]:
pdf_title.to_csv('pdf_title_cla1_cla2.csv')

In [10]:
pdf_title_cla1_cla2=pd.read_csv('pdf_title_cla1_cla2.csv', index_col=0)
pdf_title_cla1_cla2

,pdf,title,cla1,cla2
0,https://arxiv.org/pdf/1512.03385v1.pdf,Deep Residual Learning for Image Recognition,computer vision,Retinal OCT Disease Classification
1,https://arxiv.org/pdf/1409.1556v6.pdf,Very Deep Convolutional Networks for Large-Sca...,computer vision,Image Classification
2,https://arxiv.org/pdf/1704.04861v1.pdf,MobileNets: Efficient Convolutional Neural Net...,computer vision,Image Classification
3,https://arxiv.org/pdf/1608.06993v5.pdf,Densely Connected Convolutional Networks,computer vision,Crowd Counting
4,https://arxiv.org/pdf/1911.11929v1.pdf,CSPNet: A New Backbone that can Enhance Learni...,computer vision,Real-Time Object Detection
...,...,...,...,...
89,https://arxiv.org/pdf/1803.10464v2.pdf,Learning Pixel-level Semantic Affinity with Im...,computer vision,Weakly-Supervised Semantic Segmentation
90,http://openaccess.thecvf.com/content_ICCV_2019...,Integral Object Mining via Online Attention Ac...,computer vision,Weakly supervised segmentation
91,http://openaccess.thecvf.com/content_ICCV_2019...,HistoSegNet: Semantic Segmentation of Histolog...,computer vision,whole slide images
92,https://arxiv.org/pdf/2004.04581v1.pdf,Self-supervised Equivariant Attention Mechanis...,computer vision,Weakly-Supervised Semantic Segmentation


In [142]:
cla1=[]
cla2=[]

for title in pdf_title['title'].values:
    #for class2 
    if title in second_classification.title.values:
        find_cla2 = second_classification[second_classification['title'] == title]
        for cla2s in find_cla2.values:
            for cla2 in cla2s:
                
                cla2.append(cla2 for cla2)
                

Deep Residual Learning for Image Recognition
Retinal OCT Disease Classification
computer vision
Very Deep Convolutional Networks for Large-Scale Image Recognition
Image Classification
computer vision
MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications
Image Classification
computer vision
Densely Connected Convolutional Networks
Crowd Counting
computer vision
CSPNet: A New Backbone that can Enhance Learning Capability of CNN
Real-Time Object Detection
computer vision
MobileNetV2: Inverted Residuals and Linear Bottlenecks
Retinal OCT Disease Classification
computer vision
Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization
Visual Question Answering
computer vision
EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks
Fine-Grained Image Classification
computer vision
Rethinking the Inception Architecture for Computer Vision
Retinal OCT Disease Classification
computer vision
An Image is Worth 16x16 Words: Transf

In [20]:
second_classification.to_csv('second_classification-Copy1.csv')

In [29]:
second_classification_dict=second_classification.set_index('title').to_dict('index')

In [12]:
first_classification=first_classification_all.to_dict('list')

In [74]:
#first_classification에서 cla2가 중복되는 경우
cla2_second_all = second_classification['cla2']
cla2_first_list = first_classification
#cla2_second=3255개
tmp=[]
cla1_from_cla2=[]


for key, tmp_first in first_classification.items(): #16    
  
    for cla2_first in tmp_first: #782

        for cla2_second in cla2_second_all: #여기도 cla2_second=3255
            tmp.append(cla2_second)
            
            
            
            if cla2_second == cla2_first:
                if key == '':
                    print(key)
#                     raise
                    
#                 if cla2_first == '':
                    
#                     tmp.append(key)
#                     print(key)
            

            #if cla2_second == cla2_first:


In [10]:
second_classification.to_csv('second_classification-Copy1.csv')


NameError: name 'second_classification' is not defined

In [3]:
second_classification=pd.read_csv('second_classification.csv')
words_titles=pd.read_csv('word-titles.csv')

In [23]:
second_classification=pd.read_csv('second_classification.csv')
words_titles=pd.read_csv('word-titles.csv')

#cla1_cla2_title=pd.merge(left=second_classification,right=classification_without_word, how='left',on=)


for number in range(len(classification_without_word.columns)-1):
    for cla2 in second_classification['0']:
        cla1=classification_without_word[f'{number}'].str.contains(f'{cla2}')
        cla_cla2_title[cla2]=cla1

NameError: name 'classification_without_word' is not defined

In [38]:
import re
result = {}
sorted_result = {}
keywords=keywords_all['단어']
replace_txt_name=[]


for keyword in keywords:
    result[keyword] = {}
    
    for txt_file in glob('./Downloads/pdf_download/txt/*.txt'):
        txt_file_name = os.path.basename(txt_file).replace('.txt','')        
        with open(txt_file, encoding='utf-8') as file:
            sentences = file.read().lower().replace('-\n', '').replace('\n', ' ').split('.')
        sentences = ' '.join(sentences)
        count = sentences.count(keyword)        
        
        result[keyword][txt_file_name] = count              
        for pdf_name in pdf_title_cla1_cla2.pdf.values:

            if txt_file_name in pdf_name:
                classification = pdf_title_cla1_cla2[pdf_title_cla1_cla2['pdf'].str.contains(txt_file_name)]
                a = classification['cla1'].values
                b = classification['cla2'].values

                replace_txt_name.append(re.sub(r'[\.0-9a-zA-Z]+', f'{a} - {b}', txt_file_name))
                
                


        sorting = sorted(result[keyword], key= lambda x : result[keyword][x], reverse=True)[:5]
        sorted_result[keyword] = sorting

        


TypeError: must be str, not int

In [163]:


def find_classification(txt_file_name):

    for pdf_name in pdf_title_cla1_cla2.pdf.values:

        if txt_file_name in pdf_name:
            classification = pdf_title_cla1_cla2[pdf_title_cla1_cla2['pdf'].str.contains(txt_file_name)]
            a = classification['cla1'].values
            b = classification['cla2'].values
            c = f'{a} - {b}'

            return c



        


In [178]:
result_values=[]
before_last={}
count = 0
last = {}
the_result=[]

for key, values in sorted_result.items():
    for value in values:
        the_result.append(find_classification(value))
        count += 1

    last[key] = the_result
    if count == 5:
        the_result =[]
        count = 0

In [179]:
last

{'procedur': ["['computer vision'] - ['Fine-Grained Image Classification']",
  "['methodology'] - ['Meta-Learning']",
  "['methodology'] - ['Knowledge Distillation']",
  "['computer vision'] - ['OOD Detection']",
  "['natural language processing'] - ['Learning with noisy labels']"],
 'architectur': ["['computer vision'] - ['Thermal Image Segmentation']",
  "['medical'] - ['Brain Tumor Segmentation']",
  "['computer vision'] - ['Fine-Grained Image Classification']",
  "['computer vision'] - ['Fine-Grained Image Classification']",
  "['methodology'] - ['Knowledge Distillation']"],
 'imagenet': ["['methodology'] - ['Few-Shot Image Classification']",
  "['computer vision'] - ['Fine-Grained Image Classification']",
  "['methodology'] - ['Meta-Learning']",
  "['computer vision'] - ['Fine-Grained Image Classification']",
  "['computer vision'] - ['Document Layout Analysis']"],
 'augment': ["['computer vision'] - ['Image Augmentation']",
  "['computer vision'] - ['Fine-Grained Image Classifica

In [181]:
classification_last=pd.DataFrame.from_dict(dict(last), orient='index')
classification_last
classification_last.to_csv('classification_last.csv')

In [199]:
classification_last = pd.read_csv('classification_last.csv', index_col=0)
# classification_last.rename(columns={' ':'단어'}, inplace=True )
classification_last.index.names =['단어']

In [200]:
classification_last

,0,1,2,3,4
단어,,,,,
procedur,['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Meta-Learning'],['methodology'] - ['Knowledge Distillation'],['computer vision'] - ['OOD Detection'],['natural language processing'] - ['Learning w...
architectur,['computer vision'] - ['Thermal Image Segmenta...,['medical'] - ['Brain Tumor Segmentation'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Knowledge Distillation']
imagenet,['methodology'] - ['Few-Shot Image Classificat...,['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Meta-Learning'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Document Layout Analys...
augment,['computer vision'] - ['Image Augmentation'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['OOD Detection'],['computer vision'] - ['Document Layout Analys...,['methodology'] - ['Knowledge Distillation']
model,['methodology'] - ['Few-Shot Image Classificat...,['methodology'] - ['Knowledge Distillation'],['computer vision'] - ['Out-of-Distribution De...,['methodology'] - ['One-Shot Learning'],['computer vision'] - ['Out-of-Distribution De...
differ,['methodology'] - ['Few-Shot Image Classificat...,['methodology'] - ['Knowledge Distillation'],['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Meta-Learning'],['computer vision'] - ['Instance Segmentation']
tabl,['computer vision'] - ['Retinal OCT Disease Cl...,['methodology'] - ['Few-Shot Image Classificat...,['computer vision'] - ['Multi-Human Parsing'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Image Classification']
timm,['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Document Layout Analys...,['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Image Classification'],['computer vision'] - ['Weakly-Supervised Sema...
epoch,['computer vision'] - ['Fine-Grained Image Cla...,['natural language processing'] - ['Learning w...,['computer vision'] - ['Document Layout Analys...,['computer vision'] - ['Instance Segmentation'],['medical'] - ['Brain Tumor Segmentation']


In [202]:
last_last = pd.merge(left = keywords_all, right = classification_last, how='left', on = '단어')

In [203]:
last_last

,단어,의미/해석,0,1,2,3,4
0,procedur,"['In deep learning, a computer model learns to...",['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Meta-Learning'],['methodology'] - ['Knowledge Distillation'],['computer vision'] - ['OOD Detection'],['natural language processing'] - ['Learning w...
1,architectur,['The basic deep learning architecture has a f...,['computer vision'] - ['Thermal Image Segmenta...,['medical'] - ['Brain Tumor Segmentation'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Knowledge Distillation']
2,imagenet,['ImageNet is an image database organized acco...,['methodology'] - ['Few-Shot Image Classificat...,['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Meta-Learning'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Document Layout Analys...
3,augment,['Data augmentation is a set of techniques to ...,['computer vision'] - ['Image Augmentation'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['OOD Detection'],['computer vision'] - ['Document Layout Analys...,['methodology'] - ['Knowledge Distillation']
4,model,"['In deep learning, a computer model learns to...",['methodology'] - ['Few-Shot Image Classificat...,['methodology'] - ['Knowledge Distillation'],['computer vision'] - ['Out-of-Distribution De...,['methodology'] - ['One-Shot Learning'],['computer vision'] - ['Out-of-Distribution De...
5,differ,"[""Machine learning means computers learning fr...",['methodology'] - ['Few-Shot Image Classificat...,['methodology'] - ['Knowledge Distillation'],['computer vision'] - ['Fine-Grained Image Cla...,['methodology'] - ['Meta-Learning'],['computer vision'] - ['Instance Segmentation']
6,tabl,['Building a deep learning model with TensorFl...,['computer vision'] - ['Retinal OCT Disease Cl...,['methodology'] - ['Few-Shot Image Classificat...,['computer vision'] - ['Multi-Human Parsing'],['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Image Classification']
7,timm,['`timm` is a deep-learning library created by...,['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Document Layout Analys...,['computer vision'] - ['Fine-Grained Image Cla...,['computer vision'] - ['Image Classification'],['computer vision'] - ['Weakly-Supervised Sema...
8,epoch,['An epoch is a term used in machine learning ...,['computer vision'] - ['Fine-Grained Image Cla...,['natural language processing'] - ['Learning w...,['computer vision'] - ['Document Layout Analys...,['computer vision'] - ['Instance Segmentation'],['medical'] - ['Brain Tumor Segmentation']
9,paramet,['Model Parameters are properties of training ...,['methodology'] - ['One-Shot Learning'],['computer vision'] - ['OOD Detection'],['computer vision'] - ['Panoptic Segmentation'],['computer vision'] - ['Crowd Counting'],['methodology'] - ['Few-Shot Learning']


In [212]:
last_result = last_last.to_html('결과.html', encoding="utf-8-sig")